In [1]:
import networkx as nx;
import matplotlib.pyplot as plt
import pandas as pd;
import gurobipy as gp;
from gurobipy import GRB;
import csv;
import sys;
import time;
from datetime import datetime;
import math;

In [2]:
# Master


def Master(Z, bud, G, tau, s, t):
    
    M = A + 1;
    My = 1e6;
    
    m = gp.Model("Master");

    
    # Variables
    eta = m.addVar(vtype=GRB.INTEGER, lb = 0, ub = GRB.INFINITY);
    gamma = m.addVars(G.edges, vtype=GRB.BINARY);
    zBar = m.addVars(G.edges, vtype=GRB.BINARY); 
    xBar = m.addVars(G.edges, vtype=GRB.CONTINUOUS, lb = 0, ub = GRB.INFINITY);
    
    x = {};
    y = {};
    w = {};
    alpha = {};
    theta = {};
    pi = {};
    delta = {};
    v = {};
    muTau = {};
    mu = {};
    
    
    # Constraints 
    m.addConstr(gp.quicksum(gamma[i,j]*G.edges[i,j]['cost'] for i,j in G.edges) <= budget[b]);      # Feasib gamma
    
    m.addConstrs(gp.quicksum(xBar[j,i] for i in G.successors(j))
                 - gp.quicksum(xBar[i,j] for i in G.predecessors(j)) == 0 for j in G.nodes);        # Flow x_bar
    
    for i,j in G.edges:
        m.addConstr(xBar[i,j] - (1-gamma[i,j])*G.edges[i,j]['capacity'] <= 0);  
        if (G.edges[i,j]['special']==1):
            m.addConstr(xBar[i,j] >= (1/M)*zBar[i,j]);                                               # Pessimistic z
        else:
            m.addConstr(zBar[i,j] == 0);                                                             # Not in cal A
    
    
    for n in range(len(Z)):
        
        x[n] = m.addVars(G.edges, vtype=GRB.CONTINUOUS, lb = 0, ub = GRB.INFINITY);
        y[n] = m.addVars(G.edges, vtype=GRB.CONTINUOUS, lb = 0, ub = GRB.INFINITY); 
        w[n] = m.addVar(vtype=GRB.CONTINUOUS, lb = 0, ub = GRB.INFINITY); 
        alpha[n] = m.addVars(G.nodes, vtype=GRB.CONTINUOUS); 
        theta[n] = m.addVars(G.edges, vtype=GRB.CONTINUOUS, lb = 0, ub = GRB.INFINITY); 
        pi[n] = m.addVars(G.edges, vtype=GRB.CONTINUOUS, lb = -My, ub=0);
        delta[n] = m.addVar(vtype=GRB.CONTINUOUS, lb = -My, ub=0);                               
        v[n] = m.addVars(G.edges, vtype=GRB.CONTINUOUS, lb = 0, ub = GRB.INFINITY); 
        muTau[n] = m.addVars(tau, vtype=GRB.CONTINUOUS, lb = -My, ub = 0);
        mu[n] = m.addVar(vtype=GRB.CONTINUOUS, lb = -GRB.INFINITY, ub = 0);                      
      
    
        # Capacity x_bar
        
        m.addConstrs(gp.quicksum(x[n][j,i] for i in G.successors(j))                           
                     - gp.quicksum(x[n][i,j] for i in G.predecessors(j)) == 0 for j in G.nodes);           # Flow x
    
        for i,j in G.edges:
            m.addConstr(x[n][i,j] - (1-gamma[i,j])*G.edges[i,j]['capacity'] <= 0);                      # Capacity x 
            m.addConstr(v[n][i,j] <= G.edges[i,j]['capacity']*theta[n][i,j]);                           # UB v - Linearization 
            if G.edges[i,j]['special'] == 1:
                m.addConstr(x[n][i,j] + y[n][i,j] >= (1/M)*Z[n][i,j]);                                          # (19c)
                m.addConstr(alpha[n][i] - alpha[n][j] + theta[n][i,j] + pi[n][i,j] >= 0);               # (20c)
            elif i==t and j==s:
                m.addConstr(alpha[n][i] - alpha[n][j] + theta[n][i,j] >= 0);                            # (20b)
            
    
        m.addConstr(x[n][t,s] + w[n] >= xBar[t,s]);                                                           # (19d)
    
        m.addConstr(alpha[n][t] - alpha[n][s] + theta[n][t,s] + delta[n] >= 0);                                     # (20d)
    
        m.addConstr(mu[n] == gp.quicksum(pow(2, tau[u])*muTau[n][u] for u in tau));                           # (25a)
    
        m.addConstr(eta >= gp.quicksum(Z[n][i,j]*G.edges[i,j]['special'] for i,j in G.edges) +
                    gp.quicksum(v[n][i,j] for i,j in G.edges) +
                    (1/M)*gp.quicksum(Z[n][i,j]*G.edges[i,j]['special']*pi[n][i,j] for i,j in G.edges) + mu[n]); 
    
        m.addConstr(-My*gp.quicksum(y[n][i,j]*G.edges[i,j]['special'] for i,j in G.edges) - My*w[n]
                    >= gp.quicksum(v[n][i,j] for i,j in G.edges) + 
                    (1/M)*gp.quicksum(Z[n][i,j]*G.edges[i,j]['special']*pi[n][i,j] for i,j in G.edges) + mu[n]); 
    
     
    m.setObjective(eta, GRB.MINIMIZE);
    m.update();
    m.setParam("OutputFlag", 0);
    m.optimize();
    
    #for n in range(len(Z)):
    print('\nIteration ', n, ': \n');
    print('mu:', mu[n].x);
    print('w:', w[n].x);
    for i,j in G.edges:
        print('pi:', pi[n][i,j].x);
        print('x:', x[n][i,j].x);
        print('y:', y[n][i,j].x);
        print('theta:', theta[n][i,j].x);
        #for i in G.nodes:
            #print('alpha:', alpha[n][i].x);
            
    
    LB = m.objVal;
    print('LB (Master): %g' %LB);
    
    for i,j in G.edges:
        gammaStar[i,j] = gamma[i,j].x;
    
    return gammaStar, xBar[t,s].x, LB;    

In [3]:
# Sub-problem

def Sub(gammaStar, xBar_ts, G, t, s, UB):
    
   
    M = A+1;
    
    xStar = {};
    zStar = {};
    
    s = gp.Model("sub");
    
    z = s.addVars(G.edges, vtype=GRB.BINARY);
    x = s.addVars(G.edges, vtype=GRB.CONTINUOUS, lb = 0, ub = GRB.INFINITY);
    
    s.addConstrs(gp.quicksum(x[v,u] for u in G.successors(v)) 
                 - gp.quicksum(x[u,v] for u in G.predecessors(v)) == 0 for v in G.nodes);
       
    for i,j in G.edges:
        s.addConstr(x[i,j] <= G.edges[i,j]['capacity']*(1-gammaStar[i,j]));      
        if G.edges[i,j]['special'] == 1:
            s.addConstr(x[i,j] >= (1/M)*z[i,j]);
        elif i==t and j==s:
            s.addConstr(x[i,j] - xBar_ts >= 0);
        else:
            s.addConstr(z[i,j] == 0);
            
        
    s.setObjective(gp.quicksum(z[i,j] for i,j in G.edges), GRB.MAXIMIZE);
    s.update();
    s.setParam("OutputFlag", 0);
    s.optimize();
    
    objVal = s.objVal;
    UB = min(UB, objVal);
    print('UB (Sub): %g' %UB);
    
    for i,j in G.edges:
        xStar[i,j] = x[i,j].x;
        zStar[i,j] = z[i,j].x;
    
    return xStar, zStar, objVal;

In [4]:
# Main

num = 1;
budget = [0];
rate = [1];
network = {};
z = {};
xBar_ts = {};

UBs ={};
LBs = {};

# Input -- Reading the networks
for n in range(num):
    network[n] = 'Network'+str(n+1);
    
    for b in range(len(budget)):       
        
        for r in range(len(rate)):
            
            with open(network[n]+'.csv', newline='') as f:
                reader = csv.reader(f);
                row1 = next(reader);
                s = int(row1[0]);
                t = int(row1[1]);
    
                G = nx.DiGraph();
                data = pd.read_csv(network[n]+'.csv',skiprows=1, header=None);
                n_edge = len(data.index+1);
        
                for i in range(n_edge): 
                    G.add_edge(data.iat[i,0], data.iat[i,1], capacity= data.iat[i,2], 
                    cost=data.iat[i,3], special=data.iat[i,4], trafficker=data.iat[i,5], 
                    bottom=data.iat[i,6], victim=data.iat[i,7]);
                    
                for i,j in G.edges:
                    if G.edges[i,j]['special'] == 1:
                        z[i,j] = 0;
                    else:
                        z[i,j] = 0;
                        
                    if i == s:
                        G.edges[i,j]['capacity'] = math.floor(rate[r]*G.edges[i,j]['capacity']);
                    
                A = 0;
                T = 0;
                B = 0;
                V = 0;
                U = 0;

                for i,j in G.edges:
                    if G.edges[i,j]['special'] == 1:
                        A = A + 1;
                    if G.edges[i,j]['trafficker'] == 1:
                        T = T + 1;
                    elif G.edges[i,j]['bottom'] == 1:
                        B = B + 1;
                    elif G.edges[i,j]['victim'] == 1:
                        V = V + 1;
                        U = U + G.edges[i,j]['capacity'];


            # Tau for the linearization of x_{ts}*delta
            tau = []; 
            for u in range(0,math.ceil(U)):
                tau.append(u);
             
        
        # C&CG
        
            LB = 0;
            UB = 1e6;
            eps = 0.01;
            it = 1;
            
            Z = [];
            Z.append(z);
            
            X = [];
            Gamma = [];
            
            gammaStar = {};
            xBarStar = {};
            
            while (UB - LB > eps and it < 5):
                gammaStar, xBar_ts, LBs[it] = Master(Z, budget[b], G, tau, s, t);
                Gamma.append(gammaStar);
                #X.append(xBarStar);
                xStar, zStar, UB = Sub(gammaStar, xBar_ts, G, t, s, UB);
                Z.append(zStar);
                it = it + 1;

                
                
    

Set parameter Username
Academic license - for non-commercial use only - expires 2023-03-30

Iteration  0 : 

mu: 0.0
w: 0.0
pi: 0.0
x: 0.0
y: 2000000000.0
theta: 2000000000.0
pi: 0.0
x: 0.0
y: 2000000000.0
theta: 2000000000.0
pi: 0.0
x: 0.0
y: 2000000000.0
theta: 2000000000.0
pi: 0.0
x: 0.0
y: 2000000000.0
theta: 2000000000.0
pi: 0.0
x: 0.0
y: 2000000000.0
theta: 2000000000.0
pi: 0.0
x: 0.0
y: 2000000000.0
theta: 2000000000.0
pi: 0.0
x: 0.0
y: 0.0
theta: 2000000000.0
pi: 0.0
x: 0.0
y: 0.0
theta: 2000000000.0
pi: 0.0
x: 0.0
y: 0.0
theta: 2000000000.0
pi: 0.0
x: 0.0
y: 0.0
theta: 2000000000.0
pi: 0.0
x: 0.0
y: 2000000000.0
theta: 2000000000.0
LB (Master): 0
UB (Sub): 4

Iteration  1 : 

mu: 0.0
w: 0.0
pi: 0.0
x: 0.0
y: 2000000000.0
theta: 2000000000.0
pi: 0.0
x: 0.0
y: 2000000000.0
theta: 2000000000.0
pi: 0.0
x: 0.0
y: 2000000000.0
theta: 2000000000.0
pi: 0.0
x: 0.0
y: 2000000000.0
theta: 2000000000.0
pi: 0.0
x: 0.0
y: 2000000000.0
theta: 2000000000.0
pi: 0.0
x: 0.0
y: 2000000000.0
theta